# General Functions for handling different source file formats

More details can be found in the Medium article: 

## Handling .xlsb format files

In [25]:
#!pip install pyxlsb

In [26]:
import pandas as pd

In [27]:
df = pd.read_excel("./data/SAP Data Dictionary Selected Tables.xlsb", engine="pyxlsb")

In [28]:
df.head()

,Field,Field Name,Data element,Checktable,Datatype,Length,Decimals
0,MANDT,Client,MANDT,T000,CLNT,3,0
1,BUKRS,Company Code,BUKRS,T001,CHAR,4,0
2,BELNR,Accounting Document Number,BELNR_D,NaN,CHAR,10,0
3,GJAHR,Fiscal Year,GJAHR,NaN,NUMC,4,0
4,BLART,Document Type,BLART,T003,CHAR,2,0


## csv files with encoding other than utf8

In [29]:
#!pip install chardet

In [30]:
import pandas as pd

In [31]:
#Use chardet to detect the encoding in the file

import chardet

file = "./data/Oscars-demographics-DFE.csv"
with open(file, 'rb') as f:
    encoding = chardet.detect(f.read())
print(encoding)

{'encoding': 'MacRoman', 'confidence': 0.7295780722034502, 'language': ''}


In [11]:
# There will be an error reading from file 
df_utf8 = pd.read_csv(file)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xcc in position 10443: invalid continuation byte

In [32]:
# Use the same method to read based on the encoding detected earlier
df_w_encoding = pd.read_csv(file, encoding=encoding['encoding'])
df_w_encoding.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,birthplace,birthplace:confidence,date_of_birth,date_of_birth:confidence,race_ethnicity,...,award,biourl,birthplace_gold,date_of_birth_gold,movie,person,race_ethnicity_gold,religion_gold,sexual_orientation_gold,year_of_award_gold
0,670454353,False,finalized,3,2/10/15 3:45,"Chisinau, Moldova",1.0,30-Sep-1895,1.0,White,...,Best Director,http://www.nndb.com/people/320/000043191/,NaN,NaN,Two Arabian Knights,Lewis Milestone,NaN,NaN,NaN,NaN
1,670454354,False,finalized,3,2/10/15 2:03,"Glasgow, Scotland",1.0,2-Feb-1886,1.0,White,...,Best Director,http://www.nndb.com/people/626/000042500/,NaN,NaN,The Divine Lady,Frank Lloyd,NaN,NaN,NaN,NaN
2,670454355,False,finalized,3,2/10/15 2:05,"Chisinau, Moldova",1.0,30-Sep-1895,1.0,White,...,Best Director,http://www.nndb.com/people/320/000043191/,NaN,NaN,All Quiet on the Western Front,Lewis Milestone,NaN,NaN,NaN,NaN
3,670454356,False,finalized,3,2/10/15 2:04,"Chicago, Il",1.0,23-Feb-1899,1.0,White,...,Best Director,http://www.nndb.com/people/544/000041421/,NaN,NaN,Skippy,Norman Taurog,NaN,NaN,NaN,NaN
4,670454357,False,finalized,3,2/10/15 1:48,"Salt Lake City, Ut",1.0,23-Apr-1894,1.0,White,...,Best Director,http://www.nndb.com/people/292/000044160/,NaN,NaN,Bad Girl,Frank Borzage,NaN,NaN,NaN,NaN


## Excel with multiple sheet of data 

- Using a Sample Data Dictionary as example where each sheet rrepresents an SAP table which contains the data definition of the fields within
- When sheets_name = None parameter is passed in, pandas will read the data in all the sheets as a df and create a dictionary with the sheetname as key and the df as the value

In [33]:
# Reading in the data
sample_SAP_data_dict = "./data/SAP Data Dictionary Selected Tables.xlsx"
data_sheet_dict = pd.read_excel(sample_SAP_data_dict, sheet_name=None)

In [34]:
#Display the names of each sheet in excel
list_sheets = list(data_sheet_dict.keys())
print(f"There are {len(list_sheets)} sheets in the excel file")
print(f"The sheets are {list_sheets}")

There are 11 sheets in the excel file
The sheets are ['BKPF', 'BSAK', 'BSEG', 'EKPO', 'EKKO', 'EBAN', 'EKBE', 'EKKN', 'ESLL', 'ESLH', 'MAKT']


In [35]:
# Each individual df is stored within the corresponding value 
data_sheet_dict['BSEG'].head()

,Field,Field Name,Data element,Checktable,Datatype,Length,Decimals
0,MANDT,Client,MANDT,T000,CLNT,3,0
1,BUKRS,Company Code,BUKRS,T001,CHAR,4,0
2,BELNR,Accounting Document Number,BELNR_D,NaN,CHAR,10,0
3,GJAHR,Fiscal Year,GJAHR,NaN,NUMC,4,0
4,BUZEI,Number of Line Item Within Accounting Document,BUZEI,NaN,NUMC,3,0


One application of using the information is to create a table showing the common columns among each table where the tables are represented as rows and columns represented as columns

In [36]:
df = data_sheet_dict['BSEG']

In [37]:
df['Present'] = 1
df['SAP_Table'] = 'BSEG'

In [38]:
df[['SAP_Table','Field', 'Present']].pivot_table(columns='Field',
                                     values='Present',
                                     index='SAP_Table')

Field,ABPER,ABSBT,AGZEI,ALTKT,ANBWA,ANFAE,ANFBJ,ANFBN,ANFBU,ANLN1,...,ZEKKN,ZFBDT,ZINKZ,ZLSCH,ZLSPR,ZOLLD,ZOLLT,ZTERM,ZUMSK,ZUONR
SAP_Table,,,,,,,,,,,,,,,,,,,,,
BSEG,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [39]:
def extract_columns_info(df, sheet_name, list_column_field):
    df_temp = df.copy()
    df_temp['SAP_Table'] = sheet_name
    df_temp['Present'] = 1
    columns_select = ['SAP_Table'] + list_column_field + ['Present']
    print(columns_select)

    return df_temp[columns_select].pivot_table(columns=list_column_field,
                                     values='Present',
                                     index=['SAP_Table'])

In [40]:
list_column_field = ['Field']
list_df = []
for sheet_name, df in data_sheet_dict.items():
    print(df)
    print(sheet_name)
    df_processed = extract_columns_info(df,sheet_name, list_column_field)
    list_df.append(df_processed)

        Field                          Field Name Data element Checktable  \
0       MANDT                              Client        MANDT       T000   
1       BUKRS                        Company Code        BUKRS       T001   
2       BELNR          Accounting Document Number      BELNR_D        NaN   
3       GJAHR                         Fiscal Year        GJAHR        NaN   
4       BLART                       Document Type        BLART       T003   
..        ...                                 ...          ...        ...   
110     KNUMV    Number of the document condition        KNUMV        NaN   
111  OINETNUM  Exchange - netting document number   OIA_NETNUM        NaN   
112   OINJAHR                         Fiscal Year        GJAHR        NaN   
113    OININD    Movement-based netting indicator      OIA_NET        NaN   
114     RECHN             Indicator: post invoice        XRECH        NaN   

    Datatype  Length  Decimals  
0       CLNT       3         0  
1       C

   Field                                         Field Name Data element  \
0  MANDT                                             Client        MANDT   
1  MATNR                                    Material Number        MATNR   
2  SPRAS                                       Language Key        SPRAS   
3  MAKTX                  Material Description (Short Text)        MAKTX   
4  MAKTG  Material description in upper case for matchcodes        MAKTG   

  Checktable Datatype  Length  Decimals  
0       T000     CLNT       3         0  
1       MARA     CHAR      18         0  
2       T002     LANG       1         0  
3        NaN     CHAR      40         0  
4        NaN     CHAR      40         0  
MAKT
['SAP_Table', 'Field', 'Present']


In [41]:
df_consolidated = pd.concat(list_df)
df_consolidated.fillna("", inplace=True)

In [42]:
columns = ['BELNR','BUZEI','GJAHR','EBELN','EBELP','PACKNO','INTROW','MATNR']
df_consolidated.loc[:,columns]

Field,BELNR,BUZEI,GJAHR,EBELN,EBELP,PACKNO,INTROW,MATNR
SAP_Table,,,,,,,,
BKPF,1.0,,1.0,,,,,
BSAK,1.0,1.0,1.0,1.0,1.0,,,
BSEG,1.0,1.0,1.0,1.0,1.0,,,1.0
EKPO,,,,1.0,1.0,1.0,,1.0
EKKO,,,,1.0,,,,
EBAN,,,,1.0,1.0,1.0,,1.0
EKBE,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
EKKN,,,,1.0,1.0,,,
ESLL,1.0,,,,,1.0,1.0,


## Handling .txt files

In [43]:
import pandas as pd
import re

In [44]:
with open("data/Sample SAP Format.txt", encoding="utf-8") as f:
    content_raw = f.read()

In [45]:
content_raw

'DD.MM.YYYY           Dynamic List Display           1\n-----------------------------------------------------\n-----------------------------------------------------\n|DocumentNo|Year|Itm|Clrng doc.|Text                |\n-----------------------------------------------------\n|10002001  |2022|001|60007000  |Item 1              |\n|10002002  |2022|001|60007001  |Item A|B|C          |\n|10002004  |2022|001|60007006  |Item Z    \n         |\n|10002003  |2022|002|60007005  |Item A|ID:01        |\n-----------------------------------------------------\n'

In [46]:
#Alternative way to read the file with CRLF (Carriage Return Line Feed intact)
with open("data/Sample SAP Format.txt", encoding="utf-8", newline="") as f:
    content_raw_2 = f.read()

In [47]:
content_raw_2

'DD.MM.YYYY           Dynamic List Display           1\r\n-----------------------------------------------------\r\n-----------------------------------------------------\r\n|DocumentNo|Year|Itm|Clrng doc.|Text                |\r\n-----------------------------------------------------\r\n|10002001  |2022|001|60007000  |Item 1              |\r\n|10002002  |2022|001|60007001  |Item A|B|C          |\r\n|10002004  |2022|001|60007006  |Item Z    \r\n         |\r\n|10002003  |2022|002|60007005  |Item A|ID:01        |\r\n-----------------------------------------------------\r\n'

In [48]:
re.findall(r"\r\n", content_raw)

[]

In [49]:
re.findall(r"\r\n", content_raw_2)

['\r\n',
 '\r\n',
 '\r\n',
 '\r\n',
 '\r\n',
 '\r\n',
 '\r\n',
 '\r\n',
 '\r\n',
 '\r\n',
 '\r\n']

## Parquet Files

In [50]:
#!pip install pyarrow

In [52]:
import pyarrow
import pyarrow.parquet as pq

In [53]:
df_retail_original = pd.read_excel("./data/Online Retail.xlsx")

In [54]:
df_retail_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [55]:
df_retail_original.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [56]:
for column in ['InvoiceNo', 'StockCode', 'Description','Country']:
    df_retail_original[column] = df_retail_original[column].astype(str)

In [62]:
%%time
df_retail_original.to_parquet("./data/Online_Retail.parquet")

CPU times: total: 188 ms
Wall time: 270 ms


In [63]:
%%time
df_retail_original.to_csv("./data/Online_Retail.csv", index=False)

CPU times: total: 1.86 s
Wall time: 2.07 s


In [64]:
%%time
df_from_parquet = pq.read_pandas("./data/Online_Retail.parquet").to_pandas()

CPU times: total: 109 ms
Wall time: 122 ms


In [65]:
%%time
df_from_csv = pd.read_csv("./data/Online_Retail.csv")

CPU times: total: 422 ms
Wall time: 427 ms


In [66]:
parquet_file = pq.ParquetFile("./data/Online_Retail.parquet")

In [67]:
parquet_file.metadata

  created_by: parquet-cpp-arrow version 9.0.0
  num_columns: 8
  num_rows: 541909
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 4855

In [68]:
parquet_file.schema

required group field_id=-1 schema {
  optional binary field_id=-1 InvoiceNo (String);
  optional binary field_id=-1 StockCode (String);
  optional binary field_id=-1 Description (String);
  optional int64 field_id=-1 Quantity;
  optional int64 field_id=-1 InvoiceDate (Timestamp(isAdjustedToUTC=false, timeUnit=microseconds, is_from_converted_type=false, force_set_converted_type=false));
  optional double field_id=-1 UnitPrice;
  optional double field_id=-1 CustomerID;
  optional binary field_id=-1 Country (String);
}